In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import soundfile as sf

import scipy.io.wavfile as wav


import commons
import model_utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
predictor = "sdp"
n_data = 1250
n_test = 80
test = f"test 6-{n_data}-{n_test}"
with_duration_discriminator = False
step = 319500


if predictor == "sdp":
 hps = model_utils.get_hparams_from_file("./configs/ITKTTS.json")
else:
 hps = model_utils.get_hparams_from_file("./configs/ITKTTS_nosdp.json")


hps_pre = model_utils.get_hparams_from_file("configs/ljs_base.json")

## generate

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()


if with_duration_discriminator:
  _ = model_utils.load_checkpoint(f"logs/{predictor}/{n_data}/with discriminator/G_{str(step)}.pth", net_g, None)
else:
  _ = model_utils.load_checkpoint(f"logs/{predictor}/{n_data}/G_{str(step)}.pth", net_g, None)

In [45]:
texts =["Kerajaan Kutai diperkirakan sebagai kerajaan tertua di Indonesia. Kerajaan ini bercorak Hindu dan diperkirakan berdiri pada abad ke-4 Masehi di tepi Sungai Mahakam, Kalimantan Timur. Keberadaannya diketahui berdasarkan prasasti Yupa yang ditemukan di Bukit Brubus, Muara Kaman. ."]
# if n_test == 80:
#   texts = [
#       "Prinsip dan strategi yang dikemukakannya tidak terbatas pada peperangan, mereka juga relevan dalam bisnis dan berbagai bidang lainnya.",
#       "Panduan strategi militer yang terklekang oleh waktu ini diambil dari peperangan dan filosofi militer Tiongkok.",
#       "Keuntungan strategis bisa dicapai saat musuh tergoda untuk melakukan serangan berbahaya.",
#       "Setiap keputusan yang diambil seorang jenderal bergantung pada intelijen.",
#       "Kalau yang baru-baru hits dianak muda banyak coffee shop itu.",
#       "lihat situasi kondisi dulu, contohnya pantai mana aja itu?",
#       "kalau menurut kamu sendiri makanan yang sedang hits baru-baru ini apa?",
#       "baik aja, dengar-dengar liburan ini kamu pergi mantai ya?",
#       "kalo dessert kesukaan kamu sendiri apa?",
#       "wedang jahe itu minuman tradisional di Solo ya?!",
#       "iya, nah kan kamu udah berangkat sendiri.",
#       "senang banget sama itu walaupun makanannya sederhana!",
#       "tapi paling suka nasi goreng spesial.",
#       "Kebijakannya melampaui konteks perang, memberikan wawasan berharga dalam berbagai aspek kehidupan termasuk politik, bisnis, olahraga, hingga interaksi sehari-hari.",
#       "Selain itu, buku ini menawarkan panduan praktis yang bisa ditindaklanjuti.",
#       "pertama menyerang strategi atau rencana musuh, kedua memisahkan musuh dari sekutunya, dan ketiga menyerang tentara musuh.",
#       "biasanya itu yang sering dipesan sama anak muda.",
#       "Dia memberikan lima garis besar cara menggunakan api selama perang.",
#       "serang saat api mencapai puncaknya, jangan menunggu sampai api padam!",
#       "halo, pagi juga gimana kabar mu?",
#       "Daya tarik seni perang yang bertahan lama dan luas terletak pada sifatnya yang abadi.",
#       "Kamu abis makan apa ayo?!",
#       "ini aku mau bahas tentang makanan ni!",
#       "puji Tuhan aku juga baik juga.",
#       "baru ada waktu luang ini, kan aku memang sibuk ngurus kerjaan itu!",
#       "banyak biasanya aku pake bumbu racik itu.",
#       "Salah satu konsep kunci dalam The Art of War ada pada pentingnya strategi dan persiapan.",
#       "Sunzu menyoroti perlunya action plan yang terdefinisi dengan baik dan strategis, serta memperingatkan agar tidak bergantung hanya pada kekuatan saja.",
#       "alesanmu suka sama terang bulan itu apa?",
#       "Memahami karakter agen ganda bisa membantu menentukan godaan yang paling efektif.",
#       "soalnya viewnya itu bagus banget!",
#       "Sedangkan Insight Agents atau agen dalam sudah tertanam dalam struktur tentara lawan.",
#       "Hindari menghancurkan perbekalan lawan yang bisa digunakan, jangan membuang-buang makanan!",
#       "The Art of War dinilai spesial sebagai salah satu karya strategi yang paling berpengaruh ke dalam sejarah.",
#       "yang unik dari makanan hits itu apa ya?",
#       "aku tau kalo kamu suka banget sama makan.",
#       "kenapa? kalau yang daerah gunung Kidul Indrayanti itu.",
#       "Contohnya kalau berhasil menyerang dengan api, segera lanjutkan dengan serangan fisik untuk memanfaatkannya.",
#       "Memaksa pasukan berbaris sejauh 30 mil bisa mengakibatkan berbagai kerugian.",
#       "halo gimana nih Vila kabarnya?",
#       "Ajaran Sunzhu sudah mempengaruhi berbagai tingkatan militer selama bertahun-tahun.",
#       "kalo kamu suka berarti tau cara buatnya dong?!",
#       "Selain itu, Sunzu menggaris bawahi pentingnya mendapatkan informasi dan membedakan kekuatan dan kelemahan lawan.",
#       "sekitar dua setengah abad yang lalu, Sunzhu menulis The Art of War.",
#       "Prinsip-prinsip ini juga sudah diterapkan dalam politik, bisnis, dan interaksi sehari-hari.",
#       "Metode ini bergantung pada kondisi cuaca, kelembaban, serta arah dan kekuatan angin.",
#       "kalau aku sih sukanya pantai klayar ya!",
#       "kalau ikut enggak sih nanti liat sikon!",
#       "banyak yang produksi kopi kayak cappucino espresso americano sama yang lainnya gitu.",
#       "Kalau terekspos, mereka menghadapi kematian.",
#       "Perlu ada keseimbangan antara mendesak mereka untuk mendapatkan keuntungan dan memaksakan mereka secara berlebihan.",
#       "Menurut Sunzhu, strategi canggih melibatkan manipulasi seperti berpura-pura mengambil jalan tanpa tujuan.",
#       "Taktik ini bertujuan untuk membingungkan musuh dan meningkatkan kemampuan prajurit pangkat rendah untuk mengikuti perintah yang rumit dan terus berkembang.",
#       "Meskipun pendekatan ini bisa sangat efektif, pendekatan ini hanya boleh diterapkan oleh pemimpin yang berpengalaman.",
#       "Dikisahkan ketika Sunzu menghadapi tantangan melatih pasukan yang terdiri dari para selir, tugas yang tampaknya mustahil.",
#       "Awalnya saat dia memberi perintah, mereka hanya akan tertawa dan mengabaikannya.",
#       "Demi menanamkan kedisiplinan, ia mengambil keputusan berat dengan mengeksekusi dua selir kesayangan raja.",
#       "Setelah tindakan drastis ini, semua orang mengikuti perintah dengan cermat.",
#       "Sambil diam-diam mengikuti jalan yang langsung dan mempunyai tujuan.",
#       "Ia menyarankan agar tentara tetap bersatu kecuali benar-benar diperlukan untuk berpisah.",
#       "Tujuannya agar seluruh pasukan tiba di medan perang dalam kondisi optimal dan secara bersamaan.",
#       "Idealnya mencapai medan pertempuran sebelum musuh.",
#       "Sunzu menekankan keuntungan strategis dari penempatan pasukan yang tepat waktu dan terpadu.",
#       "Kadang ada situasi tertentu yang memerlukan berbagai taktik dan pilihan.",
#       "Hindari menyerang kota tertentu atau mengambil rute tertentu.",
#       "Selama perencanaan, kamu mungkin menyadari bahwa beberapa keputusan bisa berakibat buruk.",
#       "Strategi-strategi ini berada di bawah wewenang jenderal.",
#       "Ada lima hal utama yang bisa membuat seorang jenderal ragu-ragu saat mengambil keputusan taktis.",
#       "Strateginya adalah memaksa tentara musuh untuk terlibat dalam area yang menantang tersebut.",
#       "Selain memenuhi kebutuhan fisik pasukan, Sunzu menekankan pentingnya menjunjung moral dan persatuan di antara para prajurit.",
#       "Para pemimpin harus bisa menginspirasi dan mengangkat semangat pasukan mereka sambil mewaspadai tanda-tanda perselisihan atau ketidakpuasan.",
#       "Kamu perlu memastikan suplai perbekalan yang berkelanjutan.",
#       "Sun Tzu menawarkan wawasan bagi para pemimpin dalam menafsirkan medan dan keuntungan terlibat dalam pertempuran di berbagai landscape.",
#       "Pertama, aksesibel train, medan yang mudah dinavigasi oleh siapa saja.",
#       "Kedua, jika kamu berhasil mengamankan dataran tinggi dengan persediaan yang cukup sebelum lawan, kamu berada dalam posisi yang sangat menguntungkan.",
#       "Situasi di mana tidak ada pihak yang mempunyai keuntungan nyata.",
#       "Sunzu menyarankan untuk mundur dan memancing musuh maju ke medan ini.",
#       "Strategi ini memungkinkan kamu menyerang dengan keunggulan yang kamu punya.",
#       "Jenis tanah ini melibatkan tekanan dari kekuatan lawan dan medan yang kasar.",
#       "Di sini ada peluang besar untuk mendapatkan tahanan dan sumber daya, tapi sulit juga untuk mundur."

      
#   ]
# elif n_test == 50:
#    texts = [
#      "Prinsip dan strategi yang dikemukakannya tidak terbatas pada peperangan, mereka juga relevan dalam bisnis dan berbagai bidang lainnya.",
#     "Panduan strategi militer yang terklekang oleh waktu ini diambil dari peperangan dan filosofi militer Tiongkok.",
#     "Keuntungan strategis bisa dicapai saat musuh tergoda untuk melakukan serangan berbahaya.",
#     "Setiap keputusan yang diambil seorang jenderal bergantung pada intelijen.",
#     "Kalau yang baru-baru hits dianak muda banyak coffee shop itu.",
#     "lihat situasi kondisi dulu, contohnya pantai mana aja itu?",
#     "kalau menurut kamu sendiri makanan yang sedang hits baru-baru ini apa?",
#     "baik aja, dengar-dengar liburan ini kamu pergi mantai ya?",
#     "kalo dessert kesukaan kamu sendiri apa?",
#     "wedang jahe itu minuman tradisional di Solo ya?!",
#     "iya, nah kan kamu udah berangkat sendiri.",
#     "senang banget sama itu walaupun makanannya sederhana!",
#     "tapi paling suka nasi goreng spesial.",
#     "Kebijakannya melampaui konteks perang, memberikan wawasan berharga dalam berbagai aspek kehidupan termasuk politik, bisnis, olahraga, hingga interaksi sehari-hari.",
#     "Selain itu, buku ini menawarkan panduan praktis yang bisa ditindaklanjuti.",
#     "pertama menyerang strategi atau rencana musuh, kedua memisahkan musuh dari sekutunya, dan ketiga menyerang tentara musuh.",
#     "biasanya itu yang sering dipesan sama anak muda.",
#     "Dia memberikan lima garis besar cara menggunakan api selama perang.",
#     "serang saat api mencapai puncaknya, jangan menunggu sampai api padam!",
#     "halo, pagi juga gimana kabar mu?",
#     "Daya tarik seni perang yang bertahan lama dan luas terletak pada sifatnya yang abadi.",
#     "Kamu abis makan apa ayo?!",
#     "ini aku mau bahas tentang makanan ni!",
#     "puji Tuhan aku juga baik juga.",
#     "baru ada waktu luang ini, kan aku memang sibuk ngurus kerjaan itu!",
#     "banyak biasanya aku pake bumbu racik itu.",
#     "Salah satu konsep kunci dalam The Art of War ada pada pentingnya strategi dan persiapan.",
#     "Sunzu menyoroti perlunya action plan yang terdefinisi dengan baik dan strategis, serta memperingatkan agar tidak bergantung hanya pada kekuatan saja.",
#     "alesanmu suka sama terang bulan itu apa?",
#     "Memahami karakter agen ganda bisa membantu menentukan godaan yang paling efektif.",
#     "soalnya viewnya itu bagus banget!",
#     "Sedangkan Insight Agents atau agen dalam sudah tertanam dalam struktur tentara lawan.",
#     "Hindari menghancurkan perbekalan lawan yang bisa digunakan, jangan membuang-buang makanan!",
#     "The Art of War dinilai spesial sebagai salah satu karya strategi yang paling berpengaruh ke dalam sejarah.",
#     "yang unik dari makanan hits itu apa ya?",
#     "aku tau kalo kamu suka banget sama makan.",
#     "kenapa? kalau yang daerah gunung Kidul Indrayanti itu.",
#     "Contohnya kalau berhasil menyerang dengan api, segera lanjutkan dengan serangan fisik untuk memanfaatkannya.",
#     "Memaksa pasukan berbaris sejauh 30 mil bisa mengakibatkan berbagai kerugian.",
#     "halo gimana nih Vila kabarnya?",
#     "Ajaran Sunzhu sudah mempengaruhi berbagai tingkatan militer selama bertahun-tahun.",
#     "kalo kamu suka berarti tau cara buatnya dong?!",
#     "Selain itu, Sunzu menggaris bawahi pentingnya mendapatkan informasi dan membedakan kekuatan dan kelemahan lawan.",
#     "sekitar dua setengah abad yang lalu, Sunzhu menulis The Art of War.",
#     "Prinsip-prinsip ini juga sudah diterapkan dalam politik, bisnis, dan interaksi sehari-hari.",
#     "Metode ini bergantung pada kondisi cuaca, kelembaban, serta arah dan kekuatan angin.",
#     "kalau aku sih sukanya pantai klayar ya!",
#     "kalau ikut enggak sih nanti liat sikon!",
#     "banyak yang produksi kopi kayak cappucino espresso americano sama yang lainnya gitu.",
#     "Kalau terekspos, mereka menghadapi kematian."
#    ]


# elif n_test == 20:
#      texts = [
#       "Prinsip dan strategi yang dikemukakannya tidak terbatas pada peperangan, mereka juga relevan dalam bisnis dan berbagai bidang lainnya.",
#       "Panduan strategi militer yang terklekang oleh waktu ini diambil dari peperangan dan filosofi militer Tiongkok.",
#       "Keuntungan strategis bisa dicapai saat musuh tergoda untuk melakukan serangan berbahaya.",
#       "Setiap keputusan yang diambil seorang jenderal bergantung pada intelijen.",
#       "Kalau yang baru-baru hits dianak muda banyak coffe shop itu.",
#       "Lihat situasi kondisi dulu, contohnya pantai mana aja itu?",
#       "Kalau menurut kamu sendiri makanan yang sedang hits baru-baru ini apa?",
#       "Baik aja, dengar-dengar liburan ini kamu pergi mantai ya?",
#       "karena rasanya tuh manis empuk-empuk gitu kayak aku.",
#       "Wedang jahe itu minuman tradisional di Solo ya?",
#       "Iya, kamu sudah berangkat sendiri.",
#       "Senang banget sama itu walaupun makanannya sederhana!",
#       "Tapi paling suka nasi goreng spesial.",
#       "Kebijakannya melampaui konteks perang, memberikan wawasan berharga dalam berbagai aspek kehidupan termasuk politik, bisnis, olahraga, hingga interaksi sehari-hari.",
#       "Selain itu, buku ini menawarkan panduan praktis yang bisa ditindaklanjuti.",
#       "Pertama menyerang strategi atau rencana musuh, kedua memisahkan musuh dari sekutunya, dan ketiga menyerang tentara musuh.",
#       "Biasanya itu yang sering dipesan sama anak muda.",
#       "Dia memberikan lima garis besar cara menggunakan api selama perang.",
#       "Serang saat api mencapai puncaknya, jangan menunggu sampai api padam.",
#       "Halo pagi juga gimana kabarmu?",
#      ]
   
if with_duration_discriminator:
  root_folder = rf"evaluate\Resemblyzer\audio_data\{test}\vits_indonesia_{n_data}_{predictor}_duration_discriminator_{str(step)}"
  # root_folder = rf"evaluate\Resemblyzer\audio_data\{test}\vits_indonesia_{n_data}_{predictor}_duration_discriminator_{str(step)}_copy"
else:
  root_folder = rf"evaluate\Resemblyzer\audio_data\{test}\vits_indonesia_{n_data}_{predictor}_{str(step)}"

  # root_folder = rf"evaluate\Resemblyzer\audio_data\{test}\vits_indonesia_{n_data}_{predictor}_{str(step)}_copy"

torch.manual_seed(0)
torch.cuda.manual_seed(0)

for idx, text in enumerate(texts):
    if with_duration_discriminator:
      filename = f"{str(step)}_{idx+1}_{predictor}_duration_discriminator.wav"
    else:
      filename = f"{str(step)}_{idx+1}_{predictor}.wav"

    print(filename)
    with torch.no_grad():   
        x_tst = get_text(text, hps).cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([x_tst.size(1)]).cuda()
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8 ,length_scale=1)[0][0,0].cpu().float().numpy()

        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    

    # subfolder = f"{idx+1:02d}"
    # full_path = os.path.join(root_folder, subfolder)
    # os.makedirs(full_path, exist_ok=True)

    # filename = os.path.join(full_path, f"generate-{idx+1}.wav")
    # sf.write(filename, audio, hps.data.sampling_rate)
    # print(f"Saving {filename}")

319500_1_sdp.wav


In [42]:
# net_g = SynthesizerTrn(
#     len(symbols),
#     hps.data.filter_length // 2 + 1,
#     hps.train.segment_size // hps.data.hop_length,
#     **hps.model).cuda()
# _ = net_g.eval()

# step = 88500

# if with_duratio_discriminator:
#   _ = utils.load_checkpoint(f"logs/{predictor}/{n_data}/with discriminator/G_{str(step)}.pth", net_g, None)
# else:
#   _ = utils.load_checkpoint(f"logs/{predictor}/{n_data}/G_{str(step)}.pth", net_g, None)

In [43]:
# for idx, text in enumerate(texts):
#     filename = f"{str(step)}_{idx+1}_{predictor}.wav"
#     print(filename)
#     with torch.no_grad():   
#         x_tst = get_text(text, hps).cuda().unsqueeze(0)
#         x_tst_lengths = torch.LongTensor([x_tst.size(1)]).cuda()
#         audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8 ,length_scale=1)[0][0,0].cpu().float().numpy()

#         ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    